In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import torch
print(torch.__file__)


/usr/local/lib/python3.10/dist-packages/torch/__init__.py


In [ ]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
classifier('we are very happy')

[{'label': 'POSITIVE', 'score': 0.9998835325241089}]

In [ ]:
classifier('ananth is not happy')

[{'label': 'NEGATIVE', 'score': 0.9997408986091614}]

In [ ]:
classifier('my dog is bit sad and more happy')

[{'label': 'POSITIVE', 'score': 0.9985768795013428}]

In [ ]:
classifier('The meeting is scheduled for 3 PM tomorrow') disitlbert

[{'label': 'POSITIVE', 'score': 0.9864764213562012}]

In [ ]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline with the specific model
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

# Test the classifier
result = classifier("This is a great product!")
print(result)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': '5 stars', 'score': 0.8600963354110718}]


In [ ]:
result = classifier("im sad")
print(result)

[{'label': '1 star', 'score': 0.30779406428337097}]


In [ ]:
result = classifier("நான் வருத்தமாக இருக்கிறேன்")
print(result)

[{'label': '5 stars', 'score': 0.43927085399627686}]


In [ ]:
result = classifier("Je suis triste")
print(result)

[{'label': '1 star', 'score': 0.49396082758903503}]


In [ ]:
result = classifier("The product is okay, but it could be improved.")
print(result)

[{'label': '3 stars', 'score': 0.770152747631073}]


In [ ]:
from transformers import AutoTokenizer , TFAutoModelForSequenceClassification

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
result = classifier("i'm batman")
print(result)

[{'label': '5 stars', 'score': 0.3929806053638458}]


In [ ]:
result = classifier("i'm not a  batman")
print(result)

[{'label': '1 star', 'score': 0.3500869870185852}]


In [ ]:
import pandas as pd
import re
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

# Function to load and preprocess the WhatsApp chat file
def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    # Define regex for timestamp and group messages
    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        # Clean up each line by removing the timestamp and ignoring group-related messages
        line = line.strip()

        # Skip empty lines and group-related messages
        if not line or re.match(group_message_regex, line):
            continue

        # Remove timestamp
        line = re.sub(timestamp_regex, "", line).strip()

        # Check if the message has a valid format: username: message
        if ": " in line:
            user_message = line.split(": ", 1)
            if len(user_message) == 2:
                user, message = user_message
                messages.append(message)

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=["message"])
    return df

# Load the model and tokenizer for sentiment analysis
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create the sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Function to perform sentiment analysis on each message
def analyze_sentiment(df):
    sentiments = []
    for message in df['message']:
        result = classifier(message)
        sentiments.append(result[0]['label'])  # Get the sentiment label
    df['sentiment'] = sentiments
    return df

# Main function to process the chat file and perform sentiment analysis
def main(chat_file_path):
    # Load the WhatsApp chat file into a DataFrame
    df = load_chat(chat_file_path)

    # Perform sentiment analysis
    df_with_sentiments = analyze_sentiment(df)

    # Display the result
    print(df_with_sentiments)

# Run the program with the path to your WhatsApp chat file
if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"  # Provide the path to your WhatsApp chat file
    main(chat_file_path)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


                                      message sentiment
0                                     Hi guys   5 stars
1                                      hi bro   3 stars
2                       Hey! How are you all?   5 stars
3                                  Hello guys   5 stars
4                                  doing good   4 stars
..                                        ...       ...
76                                     France   5 stars
77           argerntina only for my man Messi    1 star
78                                     Great!   5 stars
79  Argentina will win the world cup for sure   5 stars
80                                   Will see   5 stars

[81 rows x 2 columns]


In [ ]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to load and preprocess the WhatsApp chat file
def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    # Define regex for timestamp and group messages
    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        line = line.strip()
        if not line or re.match(group_message_regex, line):
            continue
        line = re.sub(timestamp_regex, "", line).strip()
        if ": " in line:
            user_message = line.split(": ", 1)
            if len(user_message) == 2:
                _, message = user_message
                messages.append(message)

    df = pd.DataFrame(messages, columns=["message"])
    return df

# Text preprocessing function
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Tokenize, remove stopwords and punctuation, and lemmatize
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Convert messages to embeddings
def get_embeddings(messages, embedding_model, embedding_dim):
    embeddings = []
    for message in messages:
        tokens = preprocess_text(message)
        token_vectors = [
            embedding_model[word] for word in tokens if word in embedding_model
        ]
        if token_vectors:
            # Average the word embeddings for the sentence
            embeddings.append(np.mean(token_vectors, axis=0))
        else:
            # If no known tokens, use a zero vector
            embeddings.append(np.zeros(embedding_dim))
    return np.array(embeddings)

# Load Word2Vec and GloVe models
def load_embedding_models():
    word2vec_model = KeyedVectors.load_word2vec_format(
        'GoogleNews-vectors-negative300.bin', binary=True
    )
    glove_model = {}
    with open('/content/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.array(parts[1:], dtype='float32')
            glove_model[word] = vector
    return word2vec_model, glove_model

# Evaluate models using metrics
def evaluate_model(embeddings, labels):
    X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)
    classifier = RandomForestClassifier(random_state=42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

# Main function to process and analyze sentiment
def main(chat_file_path):
    # Load WhatsApp chat messages
    df = load_chat(chat_file_path)

    # Manually label the data for simplicity (replace this with actual labeled data)
    # For example: ['Positive', 'Negative', 'Neutral'] sentiment labels
    df['sentiment'] = ['Positive', 'Negative', 'Neutral'] * (len(df) // 3) + ['Positive'] * (len(df) % 3)

    # Load embeddings
    word2vec_model, glove_model = load_embedding_models()

    # Word2Vec embeddings
    print("\nEvaluating Word2Vec embeddings:")
    word2vec_embeddings = get_embeddings(df['message'], word2vec_model, 300)
    evaluate_model(word2vec_embeddings, df['sentiment'])

    # GloVe embeddings
    print("\nEvaluating GloVe embeddings:")
    glove_embeddings = get_embeddings(df['message'], glove_model, 300)
    evaluate_model(glove_embeddings, df['sentiment'])

# Run the program with the path to your WhatsApp chat file
if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"  # Provide the path to your WhatsApp chat file
    main(chat_file_path)


In [ ]:
import nltk
nltk.download('punkt')  # Download the punkt tokenizer
nltk.download('stopwords')  # Download stopwords
nltk.download('wordnet')  # Download WordNet for lemmatization


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
pip install --upgrade nltk


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to load and preprocess the WhatsApp chat file
def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    # Regex to clean timestamps and irrelevant messages
    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        line = line.strip()

        # Skip empty lines and group-related messages
        if not line or re.match(group_message_regex, line):
            continue

        # Remove timestamp
        line = re.sub(timestamp_regex, "", line).strip()

        # Check for valid user message format (username: message)
        if ": " in line:
            user_message = line.split(": ", 1)
            if len(user_message) == 2:
                _, message = user_message
                messages.append(message)

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=["message"])
    return df

# Preprocess messages: tokenization, lemmatization, and stopword removal
def preprocess_messages(messages):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    processed = []

    for message in messages:
        tokens = word_tokenize(message.lower())  # Tokenize and lowercase
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        processed.append(" ".join(tokens))  # Join tokens back to a single string

    return processed

# Load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype="float32")
            embeddings[word] = vector
    return embeddings

# Compute sentence embeddings using GloVe
def sentence_embedding(sentence, embeddings, embedding_dim=100):
    words = sentence.split()
    valid_embeddings = [embeddings[word] for word in words if word in embeddings]
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)  # Average embeddings for words in the sentence
    else:
        return np.zeros(embedding_dim)  # Return zero vector if no words have embeddings

# Perform sentiment analysis using GloVe embeddings
def analyze_sentiment_glove(df, embeddings, embedding_dim=100):
    # Define a few reference sentences for positive, neutral, and negative sentiments
    positive_ref = "happy great wonderful excellent positive"
    negative_ref = "sad bad terrible awful negative"
    neutral_ref = "okay average fine neutral"

    # Compute embeddings for reference sentiments
    positive_vec = sentence_embedding(positive_ref, embeddings, embedding_dim)
    negative_vec = sentence_embedding(negative_ref, embeddings, embedding_dim)
    neutral_vec = sentence_embedding(neutral_ref, embeddings, embedding_dim)

    sentiments = []

    for message in df["message"]:
        sent_vec = sentence_embedding(message, embeddings, embedding_dim)
        pos_sim = cosine_similarity([sent_vec], [positive_vec])[0][0]
        neg_sim = cosine_similarity([sent_vec], [negative_vec])[0][0]
        neu_sim = cosine_similarity([sent_vec], [neutral_vec])[0][0]

        # Determine sentiment based on highest similarity
        if pos_sim > neg_sim and pos_sim > neu_sim:
            sentiments.append("Positive")
        elif neg_sim > pos_sim and neg_sim > neu_sim:
            sentiments.append("Negative")
        else:
            sentiments.append("Neutral")

    df["sentiment"] = sentiments
    return df

# Main function to process the chat file and perform sentiment analysis
def main(chat_file_path, glove_file):
    # Load the WhatsApp chat file into a DataFrame
    df = load_chat(chat_file_path)

    # Preprocess messages
    df["processed_message"] = preprocess_messages(df["message"])

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Perform sentiment analysis
    df_with_sentiments = analyze_sentiment_glove(df, glove_embeddings)

    # Display the result
    print(df_with_sentiments)

# Run the program with the path to your WhatsApp chat file and GloVe embedding file
if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"  # Path to your WhatsApp chat file
    glove_file = "/content/glove.6B.100d.txt"          # Path to GloVe embedding file
    main(chat_file_path, glove_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                      message             processed_message  \
0                                     Hi guys                        hi guy   
1                                      hi bro                        hi bro   
2                       Hey! How are you all?                           hey   
3                                  Hello guys                     hello guy   
4                                  doing good                          good   
..                                        ...                           ...   
76                                     France                        france   
77           argerntina only for my man Messi          argerntina man messi   
78                                     Great!                         great   
79  Argentina will win the world cup for sure  argentina win world cup sure   
80                                   Will see                           see   

   sentiment  
0   Positive  
1   Negative  
2   Po

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to load and preprocess the WhatsApp chat file
def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    # Regex to clean timestamps and irrelevant messages
    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        line = line.strip()

        # Skip empty lines and group-related messages
        if not line or re.match(group_message_regex, line):
            continue

        # Remove timestamp
        line = re.sub(timestamp_regex, "", line).strip()

        # Check for valid user message format (username: message)
        if ": " in line:
            user_message = line.split(": ", 1)
            if len(user_message) == 2:
                _, message = user_message
                messages.append(message)

    # Convert to DataFrame
    df = pd.DataFrame(messages, columns=["message"])
    return df

# Preprocess messages: tokenization, lemmatization, and stopword removal
def preprocess_messages(messages):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    processed = []

    for message in messages:
        tokens = word_tokenize(message.lower())  # Tokenize and lowercase
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        processed.append(" ".join(tokens))  # Join tokens back to a single string

    return processed

# Load GloVe embeddings
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype="float32")
            embeddings[word] = vector
    return embeddings

# Compute sentence embeddings using GloVe
def sentence_embedding(sentence, embeddings, embedding_dim=100):
    words = sentence.split()
    valid_embeddings = [embeddings[word] for word in words if word in embeddings]
    if valid_embeddings:
        return np.mean(valid_embeddings, axis=0)  # Average embeddings for words in the sentence
    else:
        return np.zeros(embedding_dim)  # Return zero vector if no words have embeddings

# Perform sentiment analysis using GloVe embeddings
def analyze_sentiment_glove(df, embeddings, embedding_dim=100):
    # Define a few reference sentences for positive, neutral, and negative sentiments
    positive_ref = "happy great wonderful excellent positive"
    negative_ref = "sad bad terrible awful negative"
    neutral_ref = "okay average fine neutral"

    # Compute embeddings for reference sentiments
    positive_vec = sentence_embedding(positive_ref, embeddings, embedding_dim)
    negative_vec = sentence_embedding(negative_ref, embeddings, embedding_dim)
    neutral_vec = sentence_embedding(neutral_ref, embeddings, embedding_dim)

    sentiments = []

    for message in df["message"]:
        sent_vec = sentence_embedding(message, embeddings, embedding_dim)
        pos_sim = cosine_similarity([sent_vec], [positive_vec])[0][0]
        neg_sim = cosine_similarity([sent_vec], [negative_vec])[0][0]
        neu_sim = cosine_similarity([sent_vec], [neutral_vec])[0][0]

        # Determine sentiment based on highest similarity
        if pos_sim > neg_sim and pos_sim > neu_sim:
            sentiments.append("Positive")
        elif neg_sim > pos_sim and neg_sim > neu_sim:
            sentiments.append("Negative")
        else:
            sentiments.append("Neutral")

    df["sentiment_glove"] = sentiments
    return df

# Perform sentiment analysis using BERT
def analyze_sentiment_bert(df):
    # Load BERT model and tokenizer for sentiment analysis
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

    sentiments = []
    for message in df["message"]:
        result = classifier(message)
        label = result[0]['label']  # Get sentiment label
        if "1" in label or "2" in label:  # Map to Negative sentiment
            sentiments.append("Negative")
        elif "3" in label:  # Map to Neutral sentiment
            sentiments.append("Neutral")
        else:  # Map 4 or 5 to Positive sentiment
            sentiments.append("Positive")

    df["sentiment_bert"] = sentiments
    return df

# Main function to process the chat file and perform sentiment analysis
def main(chat_file_path, glove_file):
    # Load the WhatsApp chat file into a DataFrame
    df = load_chat(chat_file_path)

    # Preprocess messages
    df["processed_message"] = preprocess_messages(df["message"])

    # Load GloVe embeddings
    glove_embeddings = load_glove_embeddings(glove_file)

    # Perform sentiment analysis using GloVe
    df_with_glove = analyze_sentiment_glove(df.copy(), glove_embeddings)

    # Perform sentiment analysis using BERT
    df_with_bert = analyze_sentiment_bert(df.copy())

    # Combine the results
    df_combined = df_with_glove.copy()
    df_combined["sentiment_bert"] = df_with_bert["sentiment_bert"]

    # Display the result
    print(df_combined)
    return df_combined

# Run the program with the path to your WhatsApp chat file and GloVe embedding file
if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"  # Path to your WhatsApp chat file
    glove_file = "/content/glove.6B.100d.txt"          # Path to GloVe embedding file
    df_result = main(chat_file_path, glove_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

                                      message             processed_message  \
0                                     Hi guys                        hi guy   
1                                      hi bro                        hi bro   
2                       Hey! How are you all?                           hey   
3                                  Hello guys                     hello guy   
4                                  doing good                          good   
..                                        ...                           ...   
76                                     France                        france   
77           argerntina only for my man Messi          argerntina man messi   
78                                     Great!                         great   
79  Argentina will win the world cup for sure  argentina win world cup sure   
80                                   Will see                           see   

   sentiment_glove sentiment_bert  
0         Posit

In [ ]:
import pandas as pd
import re
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def load_chat(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        chat_lines = f.readlines()

    timestamp_regex = r"\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} -"
    group_message_regex = r"^(Messages and calls are end-to|You created group|You added|This message was deleted)"

    messages = []

    for line in chat_lines:
        line = line.strip()
        if not line or re.match(group_message_regex, line):
            continue
        line = re.sub(timestamp_regex, "", line).strip()
        if ": " in line:
            _, message = line.split(": ", 1)
            messages.append(message)

    return pd.DataFrame(messages, columns=["message"])

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text.lower())
    words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return words

def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

def glove_sentiment(messages, glove_embeddings):
    sentiments = []
    for message in messages:
        words = preprocess_text(message)
        vectors = [glove_embeddings[word] for word in words if word in glove_embeddings]
        if len(vectors) > 0:
            avg_vector = np.mean(vectors, axis=0)
            if avg_vector.mean() > 0.1:  # Positive threshold
                sentiment = "POSITIVE"
            elif avg_vector.mean() < -0.1:  # Negative threshold
                sentiment = "NEGATIVE"
            else:
                sentiment = "NEUTRAL"
        else:
            sentiment = "NEUTRAL"
        sentiments.append(sentiment)
    return sentiments

def load_bert_model(model_name="nlptown/bert-base-multilingual-uncased-sentiment"):
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)
    return classifier

def bert_sentiment(messages, classifier):
    sentiments = []
    for message in messages:
        result = classifier(message)
        label = result[0]['label']
        if "5 stars" in label or "4 stars" in label:
            sentiment = "POSITIVE"
        elif "3 stars" in label:
            sentiment = "NEUTRAL"
        else:
            sentiment = "NEGATIVE"
        sentiments.append(sentiment)
    return sentiments

def evaluate_consistency(glove_sentiments, bert_sentiments):
    consistent_count = sum(g == b for g, b in zip(glove_sentiments, bert_sentiments))
    consistency = consistent_count / len(glove_sentiments) if len(glove_sentiments) > 0 else 0
    return consistency

def main(chat_file_path, glove_file):
    df = load_chat(chat_file_path)
    print("Chat DataFrame Loaded:")
    print(df.head())

    print("Loading GloVe embeddings...")
    glove_embeddings = load_glove_embeddings(glove_file)

    print("Performing GloVe sentiment analysis...")
    df['sentiment_glove'] = glove_sentiment(df['message'], glove_embeddings)

    print("Loading BERT model...")
    bert_classifier = load_bert_model()

    print("Performing BERT sentiment analysis...")
    df['sentiment_bert'] = bert_sentiment(df['message'], bert_classifier)

    print("\nSentiment Analysis Results:")
    print(df[['message', 'sentiment_glove', 'sentiment_bert']])

    consistency = evaluate_consistency(df['sentiment_glove'], df['sentiment_bert'])
    print("\nConsistency between GloVe and BERT predictions: {:.2f}%".format(consistency * 100))

if __name__ == "__main__":
    chat_file_path = "/content/whatsapp grp chat.txt"
    glove_file = "/content/glove.6B.100d.txt"
    main(chat_file_path, glove_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Chat DataFrame Loaded:
                 message
0                Hi guys
1                 hi bro
2  Hey! How are you all?
3             Hello guys
4             doing good
Loading GloVe embeddings...
Performing GloVe sentiment analysis...
Loading BERT model...


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Performing BERT sentiment analysis...

Sentiment Analysis Results:
                                      message sentiment_glove sentiment_bert
0                                     Hi guys         NEUTRAL       POSITIVE
1                                      hi bro         NEUTRAL        NEUTRAL
2                       Hey! How are you all?         NEUTRAL       POSITIVE
3                                  Hello guys         NEUTRAL       POSITIVE
4                                  doing good         NEUTRAL       POSITIVE
..                                        ...             ...            ...
76                                     France         NEUTRAL       POSITIVE
77           argerntina only for my man Messi         NEUTRAL       NEGATIVE
78                                     Great!         NEUTRAL       POSITIVE
79  Argentina will win the world cup for sure         NEUTRAL       POSITIVE
80                                   Will see         NEUTRAL       POSITIVE

[81 rows

In [ ]:
!pip install langchain
!pip install langchain-groq
!pip install langchain-google-genai
!pip install streamlit
!pip install python-dotenv
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install streamlit pyngrok langchain-google-genai rouge-score nltk


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=88a3ee4e60dec4b4b126ed9652ed35c66960f3ab86f8ccb00d50ea0a1ffb51c0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,y must have been very good; why else would I have hated him so. Yes,it was heavy handed on the patriotism; but,without it,"you don't have much of a film. Watch their faces! The top three stars didn't really need much script. I would like to have seen them do this film without dialogue. If you've ever seen Ray Milland and Rita Gam in """"The Thief""""",maybe you know what I mean. When I was a kid in 1956 on my first trip to NYC,I made my Bronx uncle drive us to the foot of South Street looking for No. 66. That's when I knew that Hollywood couldn't be trusted. But I did find the river.<br /><br />I'm not giving away much of the story because I hate it when I know what happens before I see it for myself. If you have seen it,no elucidation is necessary. Just maybe,"someone who is reading these comments hasn't seen """"Pickup On South Street"""". You will like it; just don't go looking for 66 South Street in New York City.""",positive
0,This is one of those movies which get better w...,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Although I don't usually go for relentlessly h...,positive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Terrible writing, highly contrived, from a ""do...",negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Yumiko (Wakana Sakai), the pretty adopted daug...",negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Another violent, angry fantasy from Paul Verho...",negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Load a small subset of the dataset
file_path = "/content/IMDB Dataset.csv"
df = pd.read_csv(file_path)
df = df.sample(1000, random_state=42)  # Use a random subset of 1000 rows for training

# Preprocessing
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})
df = df.drop(columns=["sentiment"])
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["review"], df["label"], test_size=0.2, random_state=42
)

# Load a smaller model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenization function
def tokenize_texts(texts, labels, tokenizer, max_length=128):
    inputs = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"]
    }, tf.convert_to_tensor(labels)

# Tokenize and create TensorFlow datasets
train_encodings, train_labels = tokenize_texts(train_texts, train_labels, tokenizer)
test_encodings, test_labels = tokenize_texts(test_texts, test_labels, tokenizer)

train_dataset = tf.data.Dataset.from_tensor_slices((train_encodings, train_labels)).shuffle(100).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((test_encodings, test_labels)).batch(8)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model (1 epoch, small dataset)
history = model.fit(train_dataset, validation_data=test_dataset, epochs=1)

# Evaluate the model
results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

25/25 [==============================] - 47s 2s/step - loss: 0.4831 - accuracy: 0.7650
Test Loss: 0.4831
Test Accuracy: 0.7650


In [ ]:
!pip install tensorflow transformers scikit-learn gensim

In [ ]:
!pip install tensorflow

In [ ]:
!pip show tensorflow transformers


Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
---
Name: transformers
Version: 4.47.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-pa

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec

# Enable eager execution (fix for TensorFlow + Transformers)
tf.config.run_functions_eagerly(True)

# Load dataset
file_path = "/content/IMDB Dataset.csv"
df = pd.read_csv("/content/IMDB Dataset.csv")
df = df.sample(1000, random_state=42)  # Use a random subset of 1000 rows for quick experimentation
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})
df = df.drop(columns=["sentiment"])

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["review"], df["label"], test_size=0.2, random_state=42
)

# Initialize a results dictionary
results = {}

# ================================
# Bag of Words (BoW)
# ================================
print("Running Bag of Words...")
vectorizer = CountVectorizer(max_features=5000)
X_train_bow = vectorizer.fit_transform(train_texts).toarray()
X_test_bow = vectorizer.transform(test_texts).toarray()

bow_model = LogisticRegression(max_iter=1000)
bow_model.fit(X_train_bow, train_labels)
bow_preds = bow_model.predict(X_test_bow)

results["BoW"] = {
    "Accuracy": accuracy_score(test_labels, bow_preds),
    "Report": classification_report(test_labels, bow_preds)
}
print("BoW Completed")

# ================================
# GloVe
# ================================
print("Running GloVe...")
embedding_dim = 100
glove_file = "glove.6B.100d.txt"  # Ensure this file is downloaded and accessible
embedding_index = {}
with open(glove_file, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.array(values[1:], dtype="float32")
        embedding_index[word] = coeffs

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_texts)
X_train_glove = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=128)
X_test_glove = pad_sequences(tokenizer.texts_to_sequences(test_texts), maxlen=128)

# Prepare embedding matrix
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    if i < 5000:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Build and train model
glove_model = Sequential([
    Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], input_length=128, trainable=False),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation="sigmoid")
])
glove_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
glove_model.fit(X_train_glove, train_labels, validation_split=0.2, epochs=3, batch_size=32)

# Evaluate
glove_results = glove_model.evaluate(X_test_glove, test_labels)
results["GloVe"] = {"Accuracy": glove_results[1]}
print("GloVe Completed")

# ================================
# Word2Vec
# ================================
print("Running Word2Vec...")
sentences = [review.split() for review in train_texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Create embedding vectors
def get_word2vec_embeddings(texts, model, max_length=128):
    embeddings = np.zeros((len(texts), max_length, 100))
    for i, text in enumerate(texts):
        words = text.split()[:max_length]
        for j, word in enumerate(words):
            if word in model.wv:
                embeddings[i, j] = model.wv[word]
    return embeddings

X_train_w2v = get_word2vec_embeddings(train_texts, word2vec_model)
X_test_w2v = get_word2vec_embeddings(test_texts, word2vec_model)

word2vec_nn = Sequential([
    tf.keras.layers.Input(shape=(128, 100)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
word2vec_nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
word2vec_nn.fit(X_train_w2v, train_labels, validation_split=0.2, epochs=3, batch_size=32)

# Evaluate
word2vec_results = word2vec_nn.evaluate(X_test_w2v, test_labels)
results["Word2Vec"] = {"Accuracy": word2vec_results[1]}
print("Word2Vec Completed")

# ================================
# DistilBERT (Faster BERT)
# ================================
print("Running DistilBERT...")

# Use mixed precision training for faster computation
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Use DistilBERT model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenization function
def tokenize_texts(texts, tokenizer, max_length=128):
    inputs = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"]
    }

# Tokenize training and test datasets
train_encodings = tokenize_texts(train_texts, tokenizer)
test_encodings = tokenize_texts(test_texts, tokenizer)

# Convert labels to tensors
train_labels_tensor = tf.convert_to_tensor(train_labels.to_numpy())
test_labels_tensor = tf.convert_to_tensor(test_labels.to_numpy())

# Use a smaller batch size and fewer epochs to speed up training
batch_size = 4  # Smaller batch size for faster iteration
epochs = 1  # Reduce the number of epochs to speed up training

train_dataset = tf.data.Dataset.from_tensor_slices((train_encodings, train_labels_tensor)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_encodings, test_labels_tensor)).batch(batch_size)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()

# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (batch_inputs, batch_labels) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            outputs = model(batch_inputs, training=True)
            logits = outputs.logits
            loss = loss_fn(batch_labels, logits)

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update metric
        metric.update_state(batch_labels, logits)

    train_accuracy = metric.result()
    print(f"Training accuracy after epoch {epoch + 1}: {train_accuracy:.4f}")
    metric.reset_state()

# Evaluation
print("Evaluating DistilBERT...")
for step, (batch_inputs, batch_labels) in enumerate(test_dataset):
    outputs = model(batch_inputs, training=False)
    logits = outputs.logits
    metric.update_state(batch_labels, logits)

test_accuracy = metric.result()
print(f"DistilBERT Test Accuracy: {test_accuracy:.4f}")
results["DistilBERT"] = {"Accuracy": test_accuracy.numpy()}

# ================================
# Results Summary
# ================================
print("\nComparison Results:")
for model_name, metrics in results.items():
    print(f"{model_name}: Accuracy = {metrics['Accuracy']:.4f}")


Running Bag of Words...
BoW Completed
Running GloVe...
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


20/20 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5210 - loss: 0.7041 - val_accuracy: 0.5063 - val_loss: 0.6983
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.5483 - loss: 0.6894 - val_accuracy: 0.6313 - val_loss: 0.6628
Epoch 3/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6061 - loss: 0.6576 - val_accuracy: 0.6313 - val_loss: 0.6515
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 382ms/step - accuracy: 0.5501 - loss: 0.6720
GloVe Completed
Running Word2Vec...
Epoch 1/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 18s 887ms/step - accuracy: 0.4956 - loss: 0.7025 - val_accuracy: 0.4938 - val_loss: 0.6942
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 868ms/step - accuracy: 0.4863 - loss: 0.6936 - val_accuracy: 0.4375 - val_loss: 0.6990
Epoch 3/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 17s 873ms/step - accuracy: 0.4879 - loss: 0.6936 - val_accuracy: 0.5063 - val_loss: 0.6967
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 403ms/step - accuracy: 0.5363 - loss: 0.6876
Word2Vec Completed
Running DistilBERT...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/1
Training accuracy after epoch 1: 0.6037
Evaluating DistilBERT...
DistilBERT Test Accuracy: 0.7600

Comparison Results:
BoW: Accuracy = 0.7750
GloVe: Accuracy = 0.5950
Word2Vec: Accuracy = 0.5500
DistilBERT: Accuracy = 0.7600
